In [4]:
%pip install -r requirements.txt
#pip installs a list of libraries
#list can be found in requirements.txt

#important import names
#import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
#import seaborn as sns

  Using cached wordcloud-1.9.4-cp310-cp310-win_amd64.whl.metadata (3.5 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached fastparquet-2024.11.0-cp310-cp310-win_amd64.whl.metadata (4.3 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached cramjam-2.10.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
Using cached wordcloud-1.9.4-cp310-cp310-win_amd64.whl (299 kB)
Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Using cached fastparquet-2024.11.0-cp310-cp310-win_amd64.whl (670 kB)
Using cached cramjam-2.10.0-cp310-cp310-win_amd64.whl (1.7 MB)
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/41.3 MB ? eta -:--:--
    ---------------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\avery\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.classify import ClassifierI
nltk.download('punkt_tab')
nltk.download('tagsets')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package average

In [5]:
df_elife_train = pd.read_parquet('data/Elife/train-00000-of-00001.parquet')
#get df for development data

In [6]:
df_elife_train.head(5)
#check data exists

,article,summary,section_headings,keywords,year,title
0,"In temperate climates , winter deaths exceed s...","In the USA , more deaths happen in the winter ...","[Abstract, Introduction, Results, Discussion, ...","[epidemiology, and, global, health]",2018,National and regional seasonal dynamics of all...
1,Whether complement dysregulation directly cont...,Most people have likely experienced the discom...,"[Abstract, Introduction, Results, Discussion, ...","[microbiology, and, infectious, disease, immun...",2019,Complement and CD4+ T cells drive context-spec...
2,Variation in the presentation of hereditary im...,The immune system protects an individual from ...,"[Abstract, Introduction, Results, Discussion, ...","[microbiology, and, infectious, disease, immun...",2015,Phenotypic complementation of genetic immunode...
3,Rapid and flexible interpretation of conflicti...,The brain adapts to control our behavior in di...,"[Abstract, Introduction, Results, Discussion, ...",[neuroscience],2016,Cascade of neural processing orchestrates cogn...
4,Myosin 5a is a dual-headed molecular motor tha...,Cells use motor proteins that to move organell...,"[Abstract, Introduction, Results, Discussion, ...","[structural, biology, and, molecular, biophysics]",2015,Structural dynamics of myosin 5 during process...


In [7]:
og_summ_list = df_elife_train.summary.tolist()
#grab all summaries and put them in a list

In [8]:
test_summ = og_summ_list[28]
#genomes, DNA
#target words for summary #28

In [9]:
term_dict = {"genome" : [("complete set of genetic instructions", "NN"), ("complete sets of genetic instructions", "NNS")]}
#sample dictionairy in the form of key:lemma value:list of tuples where each tuples is a pair of layterm and POS tag

In [10]:
test_summ[:10]
#test data present -> genome/s

'Our genome'

In [12]:
def create_injections(doc):
    """A function to inject layterms. Takes in a summary and Creates parallel documents, one in raw form and 
    one containing the lemmatized version of each token"""
    lemma_doc = []
    raw_doc =[]
    doc_sents = nltk.sent_tokenize(doc)
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    for sent in doc_sents:
        #print(type(sent))
        doc = regex_token.tokenize(sent)
        doc = [word.lower() for word in doc]
        doc = pos_tag(doc)
        lemmatizer = WordNetLemmatizer() 
        doc2 = [(lemmatizer.lemmatize(word[0]), word[1]) for word in doc]
        lemma_doc.append(doc2)
        raw_doc.append(doc)
    print(len(raw_doc))
    print(len(lemma_doc))
    print(raw_doc)
    print(lemma_doc)
    #doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    #lemmatizer = WordNetLemmatizer() 
    #doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return raw_doc, lemma_doc

In [13]:
raw, lemma = create_injections(test_summ)
#check that raw data and lemmatized data are of same lenght, sentence tokenizaiton is stable

22
22
[[('our', 'PRP$'), ('genomes', 'NNS'), ('contain', 'VBP'), ('around', 'IN'), ('different', 'JJ'), ('genes', 'NNS'), ('that', 'WDT'), ('code', 'VBP'), ('for', 'IN'), ('instructions', 'NNS'), ('to', 'TO'), ('create', 'VB'), ('proteins', 'NNS'), ('and', 'CC'), ('other', 'JJ'), ('important', 'JJ'), ('molecules', 'NNS')], [('when', 'WRB'), ('changes', 'NNS'), ('or', 'CC'), ('mutations', 'NNS'), ('occur', 'VBP'), ('within', 'IN'), ('these', 'DT'), ('genes', 'NNS'), ('malfunctioning', 'VBG'), ('proteins', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('damaging', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('cell', 'NN'), ('may', 'MD'), ('be', 'VB'), ('produced', 'VBN')], [('researchers', 'NNS'), ('of', 'IN'), ('human', 'JJ'), ('genetics', 'NNS'), ('have', 'VBP'), ('tried', 'VBN'), ('to', 'TO'), ('spot', 'VB'), ('the', 'DT'), ('genetic', 'JJ'), ('mutations', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('associated', 'VBN'), ('with', 'IN'), ('illnesses', 'NNS'), ('for', 'IN'), ('example', 'NN'), ('hear

In [12]:
#This was built to prototype the find function. I'm leaving it here for clarity now but will delete later
for i in range(0, len(lemma)): #for every sentence
    for j in range(0, len(lemma[i])): # for every word in every sentence
        #print(lemma[i][j])
        if lemma[i][j][0] in term_dict: #if word in term_dict
            #print(lemma[i][j][0])
            dict_list = term_dict[lemma[i][j][0]] #get the list of values
            for k in dict_list: #loop through list
                if lemma[i][j][1] == k[1]: #if term found print its location in both docs
                    print("sentence index: ", i, "word index: ", j, k)
                    #word at S0W1 = genomes
                    #word at S3W26 = genome
                    #word at S12W9 = genome
                    #S12W22 = genome
                    #S21W17 = genome

sentence index:  0 word index:  1 ('complete sets of genetic instructions', 'NNS')
sentence index:  3 word index:  26 ('complete set of genetic instructions', 'NN')
sentence index:  12 word index:  9 ('complete set of genetic instructions', 'NN')
sentence index:  12 word index:  22 ('complete set of genetic instructions', 'NN')
sentence index:  21 word index:  17 ('complete set of genetic instructions', 'NN')


In [27]:
def find(lemma):
    """function for matching the location in the raw doc with contents of the lemma doc at the parallel location"""
    collect = []
    for sent_id in range(0, len(lemma)): #for every sentence
        for word_id in range(0, len(lemma[sent_id])): # for every word in every sentence
            #print(lemma[i][j])
            if lemma[sent_id][word_id][0] in term_dict: #if word in term_dict
                #print(lemma[i][j][0])
                dict_list = term_dict[lemma[sent_id][word_id][0]] #get the list of values
                for layterm_tag in dict_list: #loop through list
                    #print(k[1])
                    #print("GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG")
                    if lemma[sent_id][word_id][1] == layterm_tag[1]:
                        collect.append((sent_id, word_id, layterm_tag[0], layterm_tag[1]))
    return collect
                        #print("sentence index: ", i, "word index: ", j, k[0])

In [28]:
rep_list = find(lemma)
print(rep_list)

[(0, 1, 'complete sets of genetic instructions', 'NNS'), (3, 26, 'complete set of genetic instructions', 'NN'), (12, 9, 'complete set of genetic instructions', 'NN'), (12, 22, 'complete set of genetic instructions', 'NN'), (21, 17, 'complete set of genetic instructions', 'NN')]


In [15]:
def replace(rep_list, raw):
    """function to replace med term with layterm"""
    for loc in rep_list:
        sent_ind = loc[0]
        word_ind = loc[1]
        layt = loc[2]
        layt_tag = loc[3]
        raw[sent_ind][word_ind] = (layt, layt_tag)
    

In [16]:
check = replace(rep_list, raw)

In [17]:
print(raw)

[[('our', 'PRP$'), ('complete sets of genetic instructions', 'NNS'), ('contain', 'VBP'), ('around', 'IN'), ('different', 'JJ'), ('genes', 'NNS'), ('that', 'WDT'), ('code', 'VBP'), ('for', 'IN'), ('instructions', 'NNS'), ('to', 'TO'), ('create', 'VB'), ('proteins', 'NNS'), ('and', 'CC'), ('other', 'JJ'), ('important', 'JJ'), ('molecules', 'NNS')], [('when', 'WRB'), ('changes', 'NNS'), ('or', 'CC'), ('mutations', 'NNS'), ('occur', 'VBP'), ('within', 'IN'), ('these', 'DT'), ('genes', 'NNS'), ('malfunctioning', 'VBG'), ('proteins', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('damaging', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('cell', 'NN'), ('may', 'MD'), ('be', 'VB'), ('produced', 'VBN')], [('researchers', 'NNS'), ('of', 'IN'), ('human', 'JJ'), ('genetics', 'NNS'), ('have', 'VBP'), ('tried', 'VBN'), ('to', 'TO'), ('spot', 'VB'), ('the', 'DT'), ('genetic', 'JJ'), ('mutations', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('associated', 'VBN'), ('with', 'IN'), ('illnesses', 'NNS'), ('for', 'IN'), (

So the goal of this code is to do a big find and replace. We create two parallel documents, one thats the "raw" document that contains teh article split by sentence then word, where each word is tagged with its POS. The 'lemma" doc is split the same but instead of using words we use lemmas. This allows us to do a two part check to make sure we're inputing the right word in the right location, by matching on both the lemma and POS. This allows for more accurate tagging and avoids issues with homographs. Currently this code is designed to run on one single article. It needs to be optimized to run on some kind of data structure full of articles and maybe wrapped into a single function. the out put of that function needs to be a real readable summary, currently it spits out the separated tokenized pos tagged version. This needs to be stripped and joined.